In [ ]:
### Imports

In [27]:
from google.cloud import storage
from IPython.display import Audio
import os
import pandas as pd
import librosa
from itertools import islice
import tensorflow as tf
import librosa
import numpy as np

### GC storage

Initialise connection to google cloud bucket

In [28]:
storage_client = storage.Client()
bucket = storage_client.bucket('soundboard_data')

In [29]:
# audio_path = "gs://your-bucket-name/your-audio-file.mp3"

In [30]:
blobs = storage_client.list_blobs('soundboard_data')

### Classifier

Binary classification model to predict whether each time-frequency bin in the spectrogram belongs to the percussive component or not. Then we can use the predicted mask to extract the percussive elements from the original audio. Map keys to tags by converting csv to dataframe to dict. 

BIG thanks to ChatGPT

The first sample in the output is the original, the second sample is the NN's attempt at predicting where the percussion happens.

Almost no effect for any of them except for the second third, where you can hopefully hear (probably only with headphones) that the noise around the percussive elements is reduced. This is definitely something to play with.



In [32]:
# Display the i audio files
for i, blob in enumerate(blobs):
        if i>4:
            break
        print(f'File {i}: {blob.name}')
        # Download the audio file to a local file
        file_path = os.path.join('../data/temp_audio', blob.name.split('/')[-1])
        blob.download_to_filename(file_path)

        # Load the audio file with librosa
        y, sr = librosa.load(file_path, sr=44100)

        # Display and play the audio file
        #display(Audio(y, rate=sr))
        from IPython.display import Audio

        # Compute spectrogram
        n_fft = 2048
        hop_length = 512
        spec = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
        mag = np.abs(spec)

        # Define target labels
        target = np.zeros_like(mag)
        target[:512,:] = 1.0

        # Normalize magnitude component
        mag_db = librosa.power_to_db(mag)
        mag_norm = (mag_db - np.min(mag_db)) / (np.max(mag_db) - np.min(mag_db))

        # Convert to TensorFlow dataset
        dataset = tf.data.Dataset.from_tensor_slices((mag_norm.T, target.T))

        # Define model architecture
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(1025, input_shape=(1025,), activation='relu'),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(1025, activation='sigmoid')
        ])

        # Compile model
        model.compile(optimizer='adam', loss='mse')

        # Train model
        model.fit(dataset.batch(32), epochs=50)

        # Use model to separate percussive elements
        pred = model.predict(mag_norm.T)
        percussive_mag = pred.T * mag

        # Inverse STFT to obtain percussive signal
        percussive_spec = percussive_mag * np.exp(1j * np.angle(spec))
        percussive_signal = librosa.istft(percussive_spec, hop_length=hop_length)

        # Display original and percussive audio
        display(Audio(y, rate=sr))
        display(Audio(percussive_signal, rate=sr))


ValueError: ('Iterator has already started', <google.api_core.page_iterator.HTTPIterator object at 0x7fb2d79f50d0>)